In [39]:
#download from Google Drive
#https://developers.google.com/drive/api/v3/quickstart/python

from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    
    page_token = None
    while True:
        #58blast2demPAN_Slope_DEM.tif
        #response = service.files().list(q="name contains 'blast2dem' AND name contains 'Slope_DEM'",
        response = service.files().list(q="name contains '*DEM'",                                          
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
        for file in response.get('files', []):
            # Process change
            print ('Found file: %s (%s)' % (file.get('name'), file.get('id')))
        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break
        
    # Call the Drive v3 API
    results = service.files().list(
        pageSize=1000, fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])

    if not items:
        print('No files found.')
    else:
        print('Files:')
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))

            
main()

Found file: demo.ipynb (128vOMgecmVJAVHkF6NXycJiG21ZyMWY1)
Found file: Section Demo kits-Scouts-FR.pdf (0Bwnt1bGwx7sKS3FsdjJ4cnBJYWM)
Found file: Section Demo kits-Beavers-FR.pdf (0Bwnt1bGwx7sKd0x5bmtxMGdBNjg)
Found file: Section Demo kits-Beavers.pdf (0Bwnt1bGwx7sKdm9feWx2aFJzN0U)
Found file: Section Demo kits-Scouts.pdf (0Bwnt1bGwx7sKbmFDSHVPUFFaSUk)
Found file: Section Demo kits-Cubs.pdf (0Bwnt1bGwx7sKVjIxQUN1SVN2Y1U)
Found file: Section Demo kits-Cubs-FR.pdf (0Bwnt1bGwx7sKLWpLSWFkMS1tUm8)
Files:
QGISDEM (1c02jXqnTyp3lPaFj9qBa8z1NKpZIjPtC)
138 (1WQDppDx-F2zBR959Ye0BVI1PgiBH1FF_)
QGISDEM (1idIcwYrLi7dNNgWS57Ut8EeigyndOcl8)
137 (1UHvTvDIpF5ZxCZyesWVNtVTPBut_dDH1)
QGISDEM (1783Pxo7Vu0KN9gESUVuQFRt84aDM7Kxf)
136 (1--7YetVeEI0xrspOfwLci9VOwUv55kfH)
QGISDEM (1_eMQP9n5Y5L_4NdTnd8VCTEhfEQxEYRt)
135 (1CsXb1fBfJE-wRq6oPr9wdaNZtgI1yLqd)
QGISDEM (11Pkqn89L6dP5Ao_MPAm7XXeOkKu1h5gJ)
134 (1IWjyGwwCyVts4ilKbBUkLdrSxzH5dDjt)
QGISDEM (15Q4DQOLui9sildm3BDRk7h6WRI5lNHDQ)
133 (1lqNQxzyweTto5aqpdWvRu8f_1

In [25]:

# walk_gdrive.py - os.walk variation with Google Drive API

import os

from apiclient.discovery import build  # pip install google-api-python-client

FOLDER = 'application/vnd.google-apps.folder'


"""Shows basic usage of the Drive v3 API.
Prints the names and ids of the first 10 files the user has access to.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
service = build('drive', 'v3', credentials=creds)




def iterfiles(name=None, is_folder=None, parent=None, order_by='folder,name,createdTime'):
    q = []
    if name is not None:
        q.append("name = '%s'" % name.replace("'", "\\'"))
    if is_folder is not None:
        q.append("mimeType %s '%s'" % ('=' if is_folder else '!=', FOLDER))
    if parent is not None:
        q.append("'%s' in parents" % parent.replace("'", "\\'"))
    params = {'pageToken': None, 'orderBy': order_by}
    if q:
        params['q'] = ' and '.join(q)
    while True:
        response = service.files().list(**params).execute()
        print(response)
        for f in response['files']:
            yield f
        try:
            params['pageToken'] = response['nextPageToken']
        except KeyError:
            print("r")
            return(0)

def walk(top='root', by_name=False):
    if by_name:
        top, = iterfiles(name=top, is_folder=True)
    else:
        top = service.files().get(fileId=top).execute()
        if top['mimeType'] != FOLDER:
            raise ValueError('not a folder: %r' % top)
    stack = [((top['name'],), top)]
    while stack:
        path, top = stack.pop()
        dirs, files = is_file = [], []
        for f in iterfiles(parent=top['id']):
            is_file[f['mimeType'] != FOLDER].append(f)
        yield path, top, dirs, files
        if dirs:
            stack.extend((path + (d['name'],), d) for d in reversed(dirs))

for kwargs in [{'top': 'spam', 'by_name': True}, {}]:
    for path, root, dirs, files in walk(**kwargs):
        print('%s\t%d %d' % ('/'.join(path), len(dirs), len(files)))

{'kind': 'drive#fileList', 'incompleteSearch': False, 'files': []}
r


ValueError: not enough values to unpack (expected 1, got 0)

In [46]:
import io
import file
store = file.Storage('tokenWrite.json')
from googleapiclient.http import MediaIoBaseDownload
file_id = '16rSuFvh8KCzOMWggzHfxgbdxQecJ_NDB'
request = service.files().get_media(fileId=file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print ("Download %d%%." % int(status.progress() * 100))

ModuleNotFoundError: No module named 'file'

In [48]:
from __future__ import print_function
from googleapiclient.discovery import build
import io
from apiclient import http
from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/drive']

SERVICE_ACCOUNT_FILE = 'credentials.json'
FILE_ID ='16rSuFvh8KCzOMWggzHfxgbdxQecJ_NDB'


creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)


    

service = build('drive', 'v3', credentials=creds)

def download_file(service, file_id, local_fd):

  request = service.files().get_media(fileId=file_id)
  media_request = http.MediaIoBaseDownload(local_fd, request)

  while True:    
    _, done = media_request.next_chunk()

    if done:
      print ('Download Complete')
      return

file_io_base = open('izuzu.tif','wb')

download_file(service=service,file_id=FILE_ID,local_fd=file_io_base)

HttpError: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/16rSuFvh8KCzOMWggzHfxgbdxQecJ_NDB?alt=media returned "The user has not granted the app 627509309949 read access to the file 16rSuFvh8KCzOMWggzHfxgbdxQecJ_NDB.">

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [11]:
object_type = 'charcoal_hearth_hill'
object_annots_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\charcoalhearths_data\\'+object_type+'\\annots\\'
object_images_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\charcoalhearths_data\\'+object_type+'\\images\\'
object_boxes_output_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\charcoalhearths_data\\'+object_type+'\\selected_training\\'

import cv2
from xml.etree import ElementTree
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(object_annots_dir) if isfile(join(object_annots_dir, f))]

for fn in onlyfiles:
    print(fn)
    tree = ElementTree.parse(object_annots_dir+fn)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    
    fn_image = root.find('./filename').text
    print(fn_image)
    box_num=0
    for box in root.findall('.//bndbox'):
        box_num=box_num+1
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        if(ymin>ymax):
            ytemp = ymin
            ymin = ymax
            ymax=ytemp
        if(xmin>xmax):
            xtemp = xmin
            xmin = xmax
            xmax=xtemp            
        coors = [xmin, ymin, xmax, ymax]
        #print(coors)
        
        img = cv2.imread(object_images_dir+fn_image)
        crop_img = img[ymin:ymax, xmin:xmax]
        
        print(crop_img.shape)
        fntd = object_boxes_output_dir+"td_"+fn[:5]+"_b"+str(box_num)+fn_image[5:]
        print(fntd)
        cv2.imwrite(fntd,crop_img)

BlueMtHearthforShawnJeff.xml
BlueMtHearthforShawnJeff.jpg
(40, 28, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b1tHearthforShawnJeff.jpg
(39, 29, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b2tHearthforShawnJeff.jpg
(39, 28, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b3tHearthforShawnJeff.jpg
(40, 28, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b4tHearthforShawnJeff.jpg
(39, 29, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b5tHearthforShawnJeff.jpg
(40, 29, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b6tHearthforShawnJeff.jpg
(33, 26, 3)
C:\a_orgs\carleton\hist3814\R\charcoalhearths_data\charcoal_hearth_hill\selected_training\td_BlueM_b7tHearth

In [13]:
#print(construction_type)
# display image with masks and bounding boxes
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes

# train set
train_set = ObjectDataset()

object_type = 'charcoal_hearth_hill'
object_annots_dir = 'C:\\a_orgs\\carleton\\hist3814\\R\\charcoalhearths_data\\'+object_type

train_set.load_dataset(object_annots_dir, is_train=True)
train_set.prepare()
# define image id
image_id = 0
# load the image
image = train_set.load_image(image_id)
# load the masks and the class ids
mask, class_ids = train_set.load_mask(image_id)
# extract bounding boxes from the masks
bbox = extract_bboxes(mask)
# display image with masks and bounding boxes
display_instances(image, bbox, mask, class_ids, train_set.class_names)

ModuleNotFoundError: No module named 'mrcnn'